In [79]:
import pickle
import numpy as np

In [80]:
def save_lists_to_file(path):
    # Open the pickle file in binary write mode
    with open(path, 'rb') as f:
    # Load each list from the file
        NE = pickle.load(f)
        EEG_segments = pickle.load(f)
        Classes = pickle.load(f)

    return NE, EEG_segments, Classes

In [96]:
def padding_x_y(EEG_segments, Classes, named_entity_list):
    X = []
    y = []
    NE = []
    for i in range(len(EEG_segments)):
        named_entity = named_entity_list[i]
        label = Classes[i][0]
        #print(label)
        EEG_list = EEG_segments[i]
        for EEG in EEG_list:
            if EEG != []:
                X.append(EEG)
                y.append(label)
                NE.append(named_entity)
    max_seq_length = max([len(x) for x in X])
    #paddding
    for i in range(len(X)):
        padding_count = max_seq_length - len(X[i])
        #print(padding_count)
        for j in range(padding_count):
            X[i].append(np.zeros((105,8)))

    return X, y, NE


In [106]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")

    Embedded_Word_labels = []
    for word in EEG_word_level_labels:
        Embedded_Word_labels.append(word_embeddings[word])

    return Embedded_Word_labels, word_embeddings

In [97]:
#read in train and test data

train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [98]:

train_NE, train_EEG_segments, train_Classes = save_lists_to_file(train_path)
test_NE, test_EEG_segments, test_Classes = save_lists_to_file(test_path)

In [105]:

#padding
X_train, y_train, NE_list = padding_x_y(train_EEG_segments, train_Classes, train_NE)
X_test, y_test, NE_list_test = padding_x_y(test_EEG_segments, test_Classes, test_NE)

In [101]:
len(NE_list)

26061

In [104]:
len(X_train[0])

7